In [3]:
# Cell 1: Import library
import cv2
import json
import numpy as np
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from math import sqrt

# Fix OpenMP crash
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Cell 2: Polygon helper
def point_in_polygon(x, y, polygon):
    return cv2.pointPolygonTest(np.array(polygon, np.int32), (int(x), int(y)), False) >= 0

# Cell 3: Main detection with red zone
def realtime_detection_with_redzone(stream_url, zona_json):
    colors = {
        "car": (255, 0, 0),
        "truck": (255, 0, 0),
        "bus": (255, 0, 0),
        "motorcycle": (0, 255, 0),
        "bicycle": (0, 255, 0),
        "person": (0, 0, 255),
    }
    duration = {}
    updated_position = {}
    red_zone_polygons = []

    for zona_data in zona_json:
        with open(zona_data, 'r') as f:
            red_zone_polygons.append(json.load(f))

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    model = YOLO('yolo11n.pt')
    model.to(device)
    label_map = model.names

    tracker = DeepSort(max_age=30)

    cap = cv2.VideoCapture(stream_url)
    if not cap.isOpened():
        print("Error opening video stream.")
        return

    while True:
        ret, frame = cap.read()
        frame_counter = 0
        if not ret or frame is None:
            print("Stream ended or invalid frame.")
            break

        results = model(frame)[0]
        detections = []

        for box in results.boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            xyxy = box.xyxy[0].cpu().numpy()
            x1, y1, x2, y2 = xyxy
            w, h = x2 - x1, y2 - y1
            x_center, y_center = x1, y1

            detections.append(([x_center, y_center, w, h], conf, cls_id))

        try:
            tracks = tracker.update_tracks(detections, frame=frame)
        except Exception as e:
            print("Tracking error:", e)
            continue

        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            ltrb = track.to_ltrb()
            x1, y1, x2, y2 = map(int, ltrb)

            cls_id = track.det_class
            cls_conf = track.det_conf
            label = label_map.get(cls_id, 'unknown')
            if cls_conf is None or label not in colors:
                continue

            cx = int((x1 + x2) / 2) # center x
            cy = int((y1 + y2) / 2) # center y

            for red_zone_polygon in red_zone_polygons:
                in_red_zone = point_in_polygon(cx, cy, red_zone_polygon)
                if in_red_zone:
                    break

            label_text = f"{label} {track_id} {cls_conf:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), colors[label], 2)
            cv2.putText(frame, label_text, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, colors[label], 2)
            cv2.circle(frame, (cx, cy), 5, colors[label], -1)

            if in_red_zone:
                if track_id not in duration:
                    duration[track_id] = 0
                    if track_id not in updated_position:
                        updated_position[track_id] = (cx, cy)
                else:
                    # check if the vehicle is static or not (use euclidean distance)
                    if frame_counter % 30 == 0:
                        if sqrt((cx - updated_position[track_id][0]) ** 2 + (cy - updated_position[track_id][1]) ** 2) < 10:
                            duration[track_id] += 1
                        else:
                            duration[track_id] = 0
                        updated_position[track_id] = (cx, cy)
                cv2.putText(frame, f"Time {duration[track_id] // 10}s", (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        for red_zone_polygon in red_zone_polygons:
            cv2.polylines(frame, [np.array(red_zone_polygon, dtype=np.int32)], isClosed=True, color=(0, 0, 255), thickness=2)
        

        cv2.imshow(f'Deteksi CCTV Real-time - ParkLens AI - {device.upper()}', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

data_video = r"video/vietnam2.mp4"
data_zona = ["zona_vietnam1.json", "zona_vietnam2.json"]
realtime_detection_with_redzone(data_video, data_zona)


0: 384x640 2 persons, 3 cars, 3 motorcycles, 46.2ms
Speed: 1.7ms preprocess, 46.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 cars, 3 motorcycles, 77.3ms
Speed: 2.0ms preprocess, 77.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 cars, 3 motorcycles, 6.4ms
Speed: 1.1ms preprocess, 6.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 cars, 3 motorcycles, 5.6ms
Speed: 1.6ms preprocess, 5.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 cars, 3 motorcycles, 5.7ms
Speed: 1.5ms preprocess, 5.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 cars, 3 motorcycles, 5.7ms
Speed: 1.0ms preprocess, 5.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 cars, 3 motorcycles, 5.7ms
Speed: 1.2ms preprocess, 5.7ms inference, 1.0ms postproc